In [1]:
import time
import os
import subprocess
import cv2
import moviepy
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Recording Video with a webcam

In [2]:
def capture_video(folder_name, delay_s=3, video_duration_s = 5):
    folder_path = f"../data/videos/{folder_name}"
    os.makedirs(folder_path, exist_ok=True)

    print("\nTaking video in...")
    while delay_s > 0:
        print(f"{delay_s}...")
        time.sleep(1)
        delay_s -= 1

    raw_video_path = f"{folder_path}/i420.avi"
    command = f"ffmpeg -y -f v4l2 -framerate 30 -i /dev/video0 -t {video_duration_s} {raw_video_path}"
    subprocess.run(command, shell=True)

    formats = {
        "mp4.mp4": "libx264",   # MP4 format
        "divx.avi": "mpeg4",    # DivX format
        "flv.flv": "flv",       # FLV format
        "xvid.avi": "libxvid"   # Xvid format
    }

    for filename, codec in formats.items():
        output_path = f"{folder_path}/{filename}"
        command = f"ffmpeg -i {raw_video_path} -c:v {codec} {output_path}"
        subprocess.run(command, shell=True)


In [3]:
capture_video("test_1")
capture_video("test_2")


Taking video in...
3...
2...
1...


ffmpeg version n7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.1 (GCC) 20240910
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libdvdnav --enable-libdvdread --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enable-librsvg --enable-librubberband --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable


Taking video in...
3...
2...
1...


ffmpeg version n7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.1 (GCC) 20240910
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libdvdnav --enable-libdvdread --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enable-librsvg --enable-librubberband --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable

# Pricessing videos into DF

In [4]:
def get_df_info(duration = 5, fps=30):
    base_path = "../data/videos/"
    test_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    
    data = []

    for folder_name in test_folders:
        folder_path = os.path.join(base_path, folder_name)
        
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)

            cap = cv2.VideoCapture(file_path)
            cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
            _, frame= cap.read()
            ref_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            cap = cv2.VideoCapture(file_path)
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(duration*fps/3))
            _, frame= cap.read()
            cv2.imwrite(f"{folder_path}/output.jpg", frame)
            compare_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            if compare_img is not None:
                # Resize to match reference if needed
                compare_img = cv2.resize(compare_img, (ref_img.shape[1], ref_img.shape[0]))

                # Compute SSIM
                ssim_score = ssim(ref_img, compare_img)
                
                # Compute Mean Squared Error (MSE)
                mse_score = np.mean((ref_img.astype("float") - compare_img.astype("float")) ** 2)

            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path) / 1024
                if ref_img is not None:
                    height, width = ref_img.shape[:2]
                image_size = f"{width}x{height}"  # Known capture resolution
                data.append([folder_name, file, duration, fps, image_size, file_size, ssim_score, mse_score])

    # Create a global DataFrame
    df = pd.DataFrame(data, columns=["test no.", "video format", "duration", "fps", "image size", "file size", "SSIM", "MSE"])
    return df

In [5]:
df = get_df_info()
df.head(10)

,test no.,video format,duration,fps,image size,file size,SSIM,MSE
0,test_2,i420.avi,5,30,640x360,277.187500,0.976839,9.317235
1,test_2,mp4.mp4,5,30,640x360,143.185547,0.977716,9.464514
2,test_2,flv.flv,5,30,640x360,336.413086,0.977054,9.159744
3,test_2,xvid.avi,5,30,640x360,196.322266,0.956996,13.598390
4,test_2,divx.avi,5,30,640x360,293.802734,0.976674,9.222856
5,test_1,i420.avi,5,30,640x360,284.259766,0.977610,8.408090
6,test_1,mp4.mp4,5,30,640x360,137.217773,0.978833,8.688403
7,test_1,flv.flv,5,30,640x360,338.509766,0.978290,8.541246
8,test_1,xvid.avi,5,30,640x360,195.636719,0.955684,12.937765
9,test_1,divx.avi,5,30,640x360,304.433594,0.977806,8.539462


# Visualisation